In [1]:
import time
import random
import string
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import warnings
warnings.filterwarnings('ignore')
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alejo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alejo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
class Scraper:
    def __init__(self, user_input):
        self.user_input = user_input
        self.data_meli = pd.DataFrame()

    def clean_user_input(self):
        

        # Convertir texto a minusculas
        text = self.user_input.lower()

        # Quitar puntuación
        text = text.translate(str.maketrans("", "", string.punctuation))

        # Tokenizar el texto
        tokens = word_tokenize(text)

        # Quitar stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.lower() not in stop_words]

        # Join the tokens back into a cleaned text
        cleaned_text = " ".join(tokens)

        return cleaned_text

    def prepare_user_input(self):
        cleaned_name = self.user_input.replace(" ", "-").lower()
        return cleaned_name

    def scraping(self):

        cleaned_text = self.clean_user_input()
        prep_clean_text = self.prepare_user_input()
        urls = ['https://listado.mercadolibre.com.ar/' + prep_clean_text]

        page_number = 50
        for i in range(0, 5000, 50):
            urls.append(f"https://listado.mercadolibre.com.ar/{prep_clean_text}_Desde_{page_number + 1}_NoIndex_True")
            page_number += 50

        # Lista para almacenar lo escrapeado
        scraped_data = []

        # Iterar URL
        for i, url in enumerate(urls, start=1):
            # Traer el HTML de la pagina
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # agarro los posteos
            content = soup.find_all('li', class_='ui-search-layout__item')

            # sobre cada posteo se itera para traer el contenido
            for post in content:
                title = post.find('h2').text
                price = post.find('span', class_='andes-money-amount__fraction').text
                post_link = post.find("a")["href"]

                try:
                    brand = post.find('span', class_= 'ui-search-item__brand-discoverability ui-search-item__group__element').text
                except:
                    brand = '-'

                try:
                    img_link = post.find("img")["data-src"]
                except:
                    img_link = post.find("img")["src"]

                try:
                    post_rvw = post.find("span", class_='ui-search-reviews__rating-number').text
                except:
                    post_rvw = '0'

                try:
                    post_rvw_amount = post.find("span", class_='ui-search-reviews__amount').text
                except:
                    post_rvw_amount = '-'

                post_data = {
                    "title": title,
                    "brand": brand,
                    "price": price,
                    "post link": post_link,
                    "image link": img_link,
                    "review": float(post_rvw),
                    "review amount": post_rvw_amount 
                }
                scraped_data.append(post_data)

        self.data_meli = pd.DataFrame(scraped_data)



In [3]:
user_input = input()
scraper = Scraper(user_input)
scraper.scraping()


In [4]:
#scraper.data_meli.to_csv('data\meli.csv')

In [10]:
scraper.data_meli.drop(columns='image link').head()

,title,brand,price,post link,review,review amount
0,Topper Strong Pace III Hombre Adultos,TOPPER,23.750,https://www.mercadolibre.com.ar/topper-strong-...,4.6,(2239)
1,Topper X-Forcer Hombre Adultos,TOPPER,25.000,https://www.mercadolibre.com.ar/zapatillas-par...,4.7,(4377)
2,Zapatillas Topper Gondor I| Negro Monocromo En...,TOPPER,31.250,https://articulo.mercadolibre.com.ar/MLA-11614...,4.6,(440)
3,Zapatilla Blanca Simil Cuero Fackie-x Niños At...,ATOMIK,30.200,https://articulo.mercadolibre.com.ar/MLA-13957...,5.0,(5)
4,Zapatilla Negra/turquesa/rojo/azul Nasau-xz Ni...,ATOMIK,23.390,https://articulo.mercadolibre.com.ar/MLA-13957...,5.0,(4)


In [11]:
scraper.data_meli.shape

(2214, 7)